Recommendation system

In [1]:
!pip install convokit
from convokit import Corpus, download
corpus = Corpus(filename=download("movie-corpus"))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.8 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-3.0.0-py3-none-any.whl size=216707 sha256=71dbc26a9393f120284b234f7f849316668c7b91e2ced239f880d9c22b7c4523
  Stored in directory: /root/.cache/pip/wheels/c4/89/8c/2677fdb888588b6f93cb6ac86bdfb020f1f1c33e0d5525b

In [2]:
corpus.print_summary_stats()

Number of Speakers: 9035
Number of Utterances: 304713
Number of Conversations: 83097


In [3]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3156247 sha256=2353eed8a02a523746081f3ccd138d802d936b956fb25a6ca5577220a58e3734
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [4]:
import pandas as pd
from surprise import Dataset,Reader,SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [5]:
# Display basic statistics
print("Number of conversations:", len(corpus.conversations))
print("Number of users:", len(corpus.speakers))
print("Number of utterances:", len(corpus.utterances))

Number of conversations: 83097
Number of users: 9035
Number of utterances: 304713


In [6]:
# Display information about conversations
for convo_id in corpus.get_conversation_ids():
    convo = corpus.get_conversation(convo_id)
    print("Conversation ID:", convo_id)
    print("Metadata:", convo.meta)
    print("Number of utterances in conversation:", len(convo.get_utterance_ids()))
    print()

Streaming output truncated to the last 5000 lines.
Conversation ID: L655326
Metadata: ConvoKitMeta({'movie_idx': 'm606', 'movie_name': 'wild things', 'release_year': '1998', 'rating': '6.60', 'votes': '40523', 'genre': "['crime', 'mystery', 'thriller']"})
Number of utterances in conversation: 2

Conversation ID: L655322
Metadata: ConvoKitMeta({'movie_idx': 'm606', 'movie_name': 'wild things', 'release_year': '1998', 'rating': '6.60', 'votes': '40523', 'genre': "['crime', 'mystery', 'thriller']"})
Number of utterances in conversation: 4

Conversation ID: L655320
Metadata: ConvoKitMeta({'movie_idx': 'm606', 'movie_name': 'wild things', 'release_year': '1998', 'rating': '6.60', 'votes': '40523', 'genre': "['crime', 'mystery', 'thriller']"})
Number of utterances in conversation: 2

Conversation ID: L654917
Metadata: ConvoKitMeta({'movie_idx': 'm606', 'movie_name': 'wild things', 'release_year': '1998', 'rating': '6.60', 'votes': '40523', 'genre': "['crime', 'mystery', 'thriller']"})
Number

In [7]:
import pandas as pd

# Create empty lists to store data
conversation_ids = []
movie_indices = []
movie_names = []
release_years = []
ratings = []
votes = []
genres = []
num_utterances = []

# Loop through conversations and extract data
for convo_id in corpus.get_conversation_ids():
    convo = corpus.get_conversation(convo_id)

    # Extract metadata from ConvoKitMeta object
    metadata = convo.meta

    # Append data to respective lists
    conversation_ids.append(convo_id)
    movie_indices.append(metadata['movie_idx'])
    movie_names.append(metadata['movie_name'])
    release_years.append(metadata['release_year'])
    ratings.append(metadata['rating'])
    votes.append(metadata['votes'])
    genres.append(metadata['genre'])
    num_utterances.append(len(convo.get_utterance_ids()))

    # Create a DataFrame from the lists
data = {
    'Conversation ID': conversation_ids,
    'Movie Index': movie_indices,
    'Movie Name': movie_names,
    'Release Year': release_years,
    'Rating': ratings,
    'Votes': votes,
    'Genres': genres,
    'Number of Utterances': num_utterances
}

df = pd.DataFrame(data)

In [8]:
reader = Reader(rating_scale=(1, 10))

In [9]:
data=df[['Conversation ID','Movie Name','Rating']]

In [10]:
data = Dataset.load_from_df(data, reader)

In [14]:
trainset, testset = train_test_split(data, test_size=0.7,random_state=42)

In [15]:
model=SVD()

In [16]:
model.fit(trainset)

In [17]:
predictions=model.test(testset)

In [18]:
rmse=accuracy.rmse(predictions)
print(f'RMSE:{rmse:.2f}')

RMSE: 0.1451
RMSE:0.15


In [19]:
top_n = 3
user_recommendations = []

In [20]:
item_column_name = 'Movie Name'
all_item_ids = df[item_column_name].unique()

In [21]:
user_id='L236416'
for item_id in all_item_ids:
    predicted_rating = model.predict(user_id, item_id).est
    user_recommendations.append((item_id, predicted_rating))

In [22]:
user_recommendations.sort(key=lambda x: x[1], reverse=True)

In [23]:
top_n = 3
print(f'Top {top_n} recommendations for user {user_id} :')
for item_id, predicted_rating in user_recommendations[:top_n]:
  print(f'Item {item_id}: Predicted Rating ={predicted_rating:.2f}')

Top 3 recommendations for user L236416 :
Item the godfather: Predicted Rating =9.03
Item neuromancer: Predicted Rating =8.93
Item fight club: Predicted Rating =8.80
